In [20]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
from wordcloud import STOPWORDS,WordCloud
from nltk.stem import WordNetLemmatizer 
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
from nltk import corpus
from nltk import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import string
import os
import re
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [2]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
!pip install pyLDAvis 

In [3]:
#data loader class
class CsvLoader:
  def __init__(self,path,filename):
    if os.getcwd() != path:
      os.chdir(path)   
    self.path = path
    self.filename = filename 
 
  def load_csv(self):
    df = pd.read_csv(self.filename,engine='python',error_bad_lines=False)
    return df
  

In [4]:
csv_loader = CsvLoader('/content','processed_tweet_data.csv')
csv_loaded = csv_loader.load_csv()
csv_loaded.dropna(subset = ["original_text"], inplace=True)
csv_loaded

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,Fri Jun 18 17:55:49 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...","🚨Africa is ""in the midst of a full-blown third...",0.166667,0.188889,en,548.0,612.0,ketuesriche,551,351,NaN,NaN,TelGlobalHealthWHOAFRO,Mass
1,Fri Jun 18 17:55:59 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Dr Moeti is head of WHO in Africa, and one of ...",0.133333,0.455556,en,195.0,92.0,Grid1949,66,92,NaN,NaN,globalhlthtwit,"Edinburgh, Scotland"
2,Fri Jun 18 17:56:07 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Thank you @research2note for creating this ama...,0.316667,0.483333,en,2.0,1.0,LeeTomlinson8,1195,1176,NaN,red4research,NHSRDForumResearch2noteNHSRDForum,NaN
3,Fri Jun 18 17:56:10 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,0.197222,en,1580.0,899.0,RIPNY08,2666,2704,NaN,NaN,HighWireTalk,NaN
4,Fri Jun 18 17:56:20 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",I think it’s important that we don’t sell COVA...,0.280000,0.620000,en,72.0,20.0,pash22,28250,30819,NaN,NaN,PeterHotez,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6521,Sat Jun 19 07:40:29 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Australia is sending vaccines.\nAustralia is s...,0.100000,0.400000,en,78.0,12.0,ravaifiji,1577,538,NaN,NaN,pseudofijian,Fiji
6522,Sat Jun 19 07:40:29 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",The Truth Behind COVID-19 Vaccines (6) —— Bell...,-0.386111,0.622222,en,2.0,2.0,Hhuatakedownccp,697,303,NaN,DrLiMengYanCCPVirus,edendomain,"Osaka-shi Chuo, Osaka"
6524,Sat Jun 19 07:41:00 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Covid19 vaccines reach the remotest places of ...,-0.050000,0.400000,en,113.0,87.0,bitu_50,410,483,NaN,NaN,N_RamchanderRao,"Jehanabad, India"
6528,Sat Jun 19 07:41:26 +0000 2021,"<a href=""http://twitter.com/download/android"" ...","Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,0.197222,en,1595.0,908.0,wayno_af007,2224,2739,NaN,NaN,HighWireTalk,"The boro, MA"


#Data Understanding
Display information on the Data

In [5]:
class DataInfo:
  def __init__(self,df):
    self.df = df

  def general_info(self):
    return self.df.info()

In [6]:
Get_Info = DataInfo(csv_loaded)
Get_Info.general_info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3720 entries, 0 to 6530
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   created_at          3720 non-null   object 
 1   source              3720 non-null   object 
 2   original_text       3720 non-null   object 
 3   polarity            3720 non-null   float64
 4   subjectivity        3720 non-null   float64
 5   lang                3720 non-null   object 
 6   favorite_count      3720 non-null   float64
 7   retweet_count       3720 non-null   float64
 8   original_author     3720 non-null   object 
 9   followers_count     3720 non-null   int64  
 10  friends_count       3720 non-null   int64  
 11  possibly_sensitive  47 non-null     object 
 12  hashtags            2060 non-null   object 
 13  user_mentions       3720 non-null   object 
 14  place               2478 non-null   object 
dtypes: float64(4), int64(2), object(9)
memory usage: 465.0+

In [7]:

  # def remove_symbols(self,sentence_list):
  #    regrex_pattern = re.compile(pattern = "["
  #       u"\U0001F600-\U0001F64F"  # emoticons
  #       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
  #       u"\U0001F680-\U0001F6FF"  # transport & map symbols
  #       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
  #                          "]+", flags = re.UNICODE)
  #    return regrex_pattern.sub(r'',sentence_list)


#Data Preparation
Select and clean the data we are going to use for our model

In [23]:
class PreparedData:
  def __init__(self,df):
    self.df = df
      
  def clean_df(self):
    df = self.df[self.df['lang'] =="en"]
    df['original_text'] = df['original_text'].astype(str)
    df['original_text'] = df['original_text'].apply(lambda x: x.lower())
    # df['original_text']= df['original_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))
    
    return df

  def select_data(self,df):
    sel_df = self.df['original_text']
    sentence_list = sel_df.tolist()
    return sentence_list

  def tokenize_data(self,sentence_list):
    #creating sentences and words from the input tweets
    words = []
    tokenizer1 = RegexpTokenizer("[a-zA-Z0-9]+")    
    for sent in sentence_list:
      for myWord in tokenizer1.tokenize(sent):
        words.append(myWord.lower())   
    return words 

  def lemmatize_data(self,word):
    wl = WordNetLemmatizer()
    return str(wl.lemmatize(word))

  def remove_stop_words(self,word):
    s_words = stopwords.words('english')
    if word not in s_words:
      return word

  def remove_none(self,wordlist):
    cleaned_list = []
    for w in wordlist:
      if w is not None:
        cleaned_list.append(w)
    return cleaned_list
  
  

In [24]:
def prepare_data(df):
    PreparedData_object = PreparedData(df)
    cleaned_df = PreparedData_object.clean_df()
    selected = PreparedData_object.select_data(cleaned_df)
    word_list = PreparedData_object.tokenize_data(selected)
    prepared_wordList = [PreparedData_object.remove_stop_words(PreparedData_object.lemmatize_data(word)) for word in word_list]
    prepared = PreparedData_object.remove_none(prepared_wordList)
    return prepared,selected  

    

In [30]:
prepared_wordlist,prepared_sentencelist = prepare_data(csv_loaded)

In [31]:
prepared_wordlist[:10]

['africa',
 'midst',
 'full',
 'blown',
 'third',
 'wave',
 'coronavirus',
 'head',
 'whoafro',
 'ha']

#Modeling
Selecting the right model and assessing it 

In [32]:
word_id = corpora.Dictionary([prepared_wordlist])
myCorpus = [word_id.doc2bow(prepared_word) for prepared_word in [prepared_wordlist]]
# Modeling
model = gensim.models.ldamodel.LdaModel(myCorpus,
                                           id2word=word_id,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [33]:
pprint(model.show_topics(formatted=False))

[(0,
  [('vaccine', 0.010447516),
   ('http', 0.006845243),
   ('co', 0.0057316404),
   ('covid19', 0.005173532),
   ('africa', 0.0032421679),
   ('world', 0.0021081867),
   ('amp', 0.0018113299),
   ('dos', 0.0014354363),
   ('need', 0.0014221477),
   ('wave', 0.0013317144)]),
 (1,
  [('vaccine', 0.0043484517),
   ('http', 0.0041330243),
   ('co', 0.0033837405),
   ('covid19', 0.002858383),
   ('africa', 0.0016686914),
   ('covid', 0.0012082566),
   ('world', 0.00109886),
   ('dos', 0.0010873323),
   ('amp', 0.001030495),
   ('health', 0.00094260153)]),
 (2,
  [('vaccine', 0.0016380927),
   ('covid19', 0.0012531837),
   ('http', 0.0011097183),
   ('co', 0.0009871629),
   ('africa', 0.00070974627),
   ('amp', 0.0004854707),
   ('world', 0.0004327206),
   ('need', 0.00041754084),
   ('dos', 0.00041509373),
   ('ha', 0.00039204842)]),
 (3,
  [('vaccine', 0.010832202),
   ('http', 0.01029136),
   ('co', 0.0098110745),
   ('covid19', 0.008276451),
   ('africa', 0.0052778944),
   ('amp', 0.

In [34]:
# Evaluation using coherence
lda_coherence_mod = CoherenceModel(model=model, texts=[prepared_wordlist], dictionary=word_id, coherence='c_v')
coherence = lda_coherence_mod.get_coherence()
print(f'Coherence score and accuracy {coherence}', )

Coherence score and accuracy 0.29324383799962356


In [35]:
pprint(model.show_topics(formatted=False))

[(0,
  [('vaccine', 0.010447516),
   ('http', 0.006845243),
   ('co', 0.0057316404),
   ('covid19', 0.005173532),
   ('africa', 0.0032421679),
   ('world', 0.0021081867),
   ('amp', 0.0018113299),
   ('dos', 0.0014354363),
   ('need', 0.0014221477),
   ('wave', 0.0013317144)]),
 (1,
  [('vaccine', 0.0043484517),
   ('http', 0.0041330243),
   ('co', 0.0033837405),
   ('covid19', 0.002858383),
   ('africa', 0.0016686914),
   ('covid', 0.0012082566),
   ('world', 0.00109886),
   ('dos', 0.0010873323),
   ('amp', 0.001030495),
   ('health', 0.00094260153)]),
 (2,
  [('vaccine', 0.0016380927),
   ('covid19', 0.0012531837),
   ('http', 0.0011097183),
   ('co', 0.0009871629),
   ('africa', 0.00070974627),
   ('amp', 0.0004854707),
   ('world', 0.0004327206),
   ('need', 0.00041754084),
   ('dos', 0.00041509373),
   ('ha', 0.00039204842)]),
 (3,
  [('vaccine', 0.010832202),
   ('http', 0.01029136),
   ('co', 0.0098110745),
   ('covid19', 0.008276451),
   ('africa', 0.0052778944),
   ('amp', 0.

Visualization

In [36]:
#Visualizing
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(model, myCorpus, word_id)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.207139  0.000002       1        1  99.998471
3     -0.023610 -0.000147       2        1   0.000272
0      0.003026  0.000369       3        1   0.000266
1      0.037550  0.000160       4        1   0.000257
5      0.038412 -0.000471       5        1   0.000257
4      0.066599 -0.000019       6        1   0.000244
2      0.085162  0.000107       7        1   0.000233, topic_info=             Term         Freq        Total Category  logprob  loglift
4940      vaccine  4236.000000  4236.000000  Default  30.0000  30.0000
2392         http  3552.000000  3552.000000  Default  29.0000  29.0000
1141           co  3553.000000  3553.000000  Default  28.0000  28.0000
1306      covid19  2785.000000  2785.000000  Default  27.0000  27.0000
487        africa  1441.000000  1441.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
3486  palestinian     0.000073   322.654937   Topic7  -8.0776  -2.3290
3748   production     0.000069   220.523793   Topic7  -8.1292  -2.0000
4569       system     0.000071   308.584803   Topic7  -8.1003  -2.3071
4932   vaccinated     0.000074   442.053140   Topic7  -8.0684  -2.6346
92              2     0.000072   330.046880   Topic7  -8.0986  -2.3727

[481 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
25        1  0.879818   0vexmme0cd
26        1  0.880054   0yec9c1o1m
28        1  1.000055            1
67        1  0.881432          172
76        1  0.999449           19
...     ...       ...          ...
5221      1  0.880332         yale
5238      1  0.880457   ykbo781dn3
5239      1  0.880707   ykhpxdw3vd
5276      1  0.881473  zelenskyyua
5282      1  0.881426   zi4xn9ff47

[209 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 1, 2, 6, 5, 3])